In [1]:
import sys
sys.path.append("../src")
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import time

# Utils

In [2]:
private_key, public_key = utils.generate_nostr_keypair()
e = utils.generate_event(private_key, public_key, int(time.time()), 1, [], "test")
assert utils.verify_sig(e['id'], e['pubkey'], e['sig']), "Signature verification failed"
assert utils.calc_event_id(e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content']) == e['id'], "Event ID calculation failed"
e = Event(e['id'], e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content'], e['sig'])
e

Event(id=0b7c22b32ef07e31468b1c509978f440509a4feb61fe667e3bb14a9d6022391c, pubkey=c610d9ae4c19556180da31a10c21838560264f0e5438ca3b220244cae2907c60, created_at=1747244439, kind=1, tags=[], content=test, sig=e488c99d1b9d838a7bca8eba3dc7cd41e5279480dcfe90e424e4fad7ffb4dd6e571480619467a00f9af0058f82c28521632de1ce2482e02a61414a0765a8cddb)

# Database

In [3]:
bigbrotr = Bigbrotr(
    host="localhost",
    port=5432,
    user="admin",
    password="admin",
    dbname="bigbrotr"
)

In [4]:
bigbrotr.connect()

In [5]:
query = "SELECT * FROM relays"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
for row in rows:
    print(row)

('wss://filter.nostr.wine/lnurl1dp68gurn8ghj7ampd3kx2ar0veekzar0wd5xjtnrdakj7tnhv4kxctttdehhwm30d3h82unvwqhkwunpw35hxcnfwf6xsv3nlmld66', 'clearnet')
('wss://filter.nostr.wine/nprofile1qqsf7dlju0m0nfcwrdl0f9upt5ktp6uuqaxtmmwykgd3c66wwlzj9kspzpmhxue69uhkummnw3ezuamfdejsacy5p8', 'clearnet')
('wss://fittest-burned-violet.blast-mainnet.quiknode.pro/9cde73cf5796fa7a5b597c7b3f597cdd4e2e2b3c', 'clearnet')
('wss://proxy.nostr-relay.app/ae30ba2372ced6c1960d49c0266fdd3e07cea52b1bf378fdb684aee6363dec46', 'clearnet')
('wss://filter.nostr.wine/3ba9b8cf58082bd37eec18455b26bb04a47f4a8e835ac18c7ea4348673ee1623', 'clearnet')
('wss://filter.nostr.wine/78b3c1ed0a53b072fcfb8cc2e2e09cad31c9bfec869d1c8745c343d55033eea9', 'clearnet')
('wss://filter.nostr.wine/4c800257a588a82849d049817c2bdaad984b25a45ad9f6dad66e47d3b47e3b2f', 'clearnet')
('wss://relay.nostr.band/93f2deb025a6923c32ea998efdefcf85781fc8961b7ade6dbf03f43625221768', 'clearnet')
('wss://g6rjgzhh7ielq4hjfdrbb3l3c73hj2i2arbkv6benaowlmfsywswktid.onion:

In [6]:
bigbrotr.close()

In [ ]:
import pandas as pd
relays_url = pd.read_csv("../seed/relays_url.csv")
relays_url = relays_url.groupby('relay_url').agg({'count': 'sum'}).reset_index()
relays_url = relays_url.sort_values(by='count', ascending=False).reset_index(drop=True)
relays_url.to_csv("../seed/relays_url.csv", index=False)

In [50]:
from aiohttp import ClientSession
from aiohttp_socks import ProxyConnector

proxy_url = f"socks5://localhost:9050"
relay = Relay("wss://relay.damus.io")
async def compute_relay_metadata(relay, proxy_url, timeout=10):
    headers = {'Accept': 'application/nostr+json'}
    relay_id = relay.url.removeprefix('wss://')
    nip11_response = None
    connector = connector = ProxyConnector.from_url(proxy_url) if relay.network == 'tor' else None
    async with ClientSession(connector=connector) as session:
        for schema in ['https://', 'http://']:
            try:
                async with session.get(schema+relay_id, headers=headers, timeout=timeout) as response:
                    if response.status == 200:
                        nip11_response = await response.json()
                        break
                    else:
                        print(f"Failed to fetch metadata from {schema+relay_id}: {response.status}")
            except Exception as e:
                print(f"Error fetching metadata from {schema+relay_id}: {e}")
    relay_metadata = {
        'name': nip11_response.get('name'),
        'description': nip11_response.get('description'),
        'banner': nip11_response.get('banner'),
        'icon': nip11_response.get('icon'),
        'pubkey': nip11_response.get('pubkey'),
        'contact': nip11_response.get('contact'),
        'supports_nips': nip11_response.get('supported_nips'),
        'software': nip11_response.get('software'),
        'version': nip11_response.get('version'),
        'privacy_policy': nip11_response.get('privacy_policy'),
        'terms_of_service': nip11_response.get('terms_of_service'),
        'limitation': nip11_response.get('limitation'),
        'extra_fields': {
            key: value 
            for key, value in nip11_response.items() 
            if key not in [
                'name', 'description', 'banner', 'icon', 'pubkey', 'contact',
                'supported_nips', 'software', 'version', 'privacy_policy',
                'terms_of_service', 'limitation'
            ]
        }
    }
    return relay_metadata

await compute_relay_metadata(relay, proxy_url)

{'name': 'damus.io',
 'description': 'Damus strfry relay',
 'banner': None,
 'icon': 'https://damus.io/img/logo.png',
 'pubkey': '32e1827635450ebb3c5a7d12c1f8e7b2b514439ac10a67eef3d9fd9c5c68e245',
 'contact': 'jb55@jb55.com',
 'supports_nips': [1, 2, 4, 9, 11, 22, 28, 40, 70, 77],
 'software': 'git+https://github.com/hoytech/strfry.git',
 'version': '1.0.4-1-g783f9ce8cc77',
 'privacy_policy': None,
 'terms_of_service': None,
 'limitation': {'max_limit': 500,
  'max_message_length': 1000000,
  'max_subscriptions': 300},
 'extra_fields': {'negentropy': 1}}